## Init e dependencies

In [ ]:
! pip install langchain==0.2.7 langchain_aws langchain-community langchain_core boto3 botocore


## LLM endpoints

In [4]:
import boto3
import os
from langchain_aws import ChatBedrock
from botocore.config import Config
import urllib3
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from google.colab import userdata
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

bedrock_client = boto3.client(
        service_name='bedrock-runtime',
        region_name='eu-west-1',
        aws_access_key_id=userdata.get('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=userdata.get('AWS_SECRET_ACCESS_KEY'),
        verify=False,  # Disable SSL verification
        config=Config(
            proxies={'https': None}
        )
    )

llm = ChatBedrock(
        model_id="anthropic.claude-3-haiku-20240307-v1:0",
        client=bedrock_client,
        model_kwargs={
            "temperature": 0,
            "max_tokens": 2000,
        }
    )


## While True

Il costrutto **while True** è una struttura di controllo utilizzata nei linguaggi di programmazione per creare loop (cicli) infiniti. In Python, un ciclo while esegue ripetutamente un blocco di codice finché la condizione specificata è vera.

Nel caso di while True, la condizione è **sempre vera**, quindi il ciclo continua all'infinito finché non viene interrotto manualmente.

In questo caso, il ciclo viene utilizzato per richiedere all'utente (tramite comando **input**) un messaggio da girare alla LLM.

Nel momento in cui l'utente inserisce una parola chiave, si usa il **break** per uscire dal ciclo infinito.

In [ ]:
print("Chatbot per l'ordine - Digita 'esci' per uscire")
while True:
    comando = input("Scrivi 'esci' per terminare: ")
    print(f"hai scritto {comando}")
    if comando.lower() == "esci":
        print("Ciclo terminato.")
        break

print("Fuori dal ciclo")

## Memory

La **ConversationBufferMemory** è un meccanismo fornito da LangChain che tiene traccia delle interazioni passate tra utente e assistente (cioè la cronologia della chat). In pratica, **memorizza** gli ultimi **input** (domande dell’utente) e **output** (risposte del bot).

Questo è utile perché consente al modello di **mantenere il contesto** delle conversazioni precedenti, dando risposte più coerenti e pertinenti mano a mano che la chat procede.

In [6]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.memory import ConversationBufferMemory
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

memory = ConversationBufferMemory(return_messages=True)
output_parser = StrOutputParser()

system_message = """
        Sei un assistente virtuale
        """

In [7]:
print("Chatbot - Digita 'esci' per uscire")

def chat_loop():
  while True:
      user_input = input("================> Tu: ")
      if user_input.lower() == "esci":
          break

      # Recupero della conversazione passata
      chat_history = memory.load_memory_variables({}).get("history", [])

      # Formattazione del prompt
      messages = [SystemMessage(content=system_message)]
      for msg in chat_history:
        print(f"HISTORY: {msg}")
        messages.append(msg)

      messages.append(HumanMessage(content=user_input))

      # Ottenere la risposta del modello
      prompt = ChatPromptTemplate.from_messages(messages)
      chain = prompt | llm | output_parser
      response = chain.invoke({})

      # Salvare il contesto nella memoria
      memory.save_context({"input": user_input}, {"output": response})

      print("Bot:", response)

Chatbot - Digita 'esci' per uscire


In [ ]:
chat_loop()

## Esercitazione

### Modificare il chat loop per implementare un assistente AI interattivo

#### Esempi

#### - Gestione Ordini (E-commerce)
Obiettivo: Permettere all’utente di visualizzare, modificare o tracciare i propri ordini su un e-commerce.
Esempio:

    Utente: «Voglio controllare lo stato del mio ordine #12345.»
    Assistente: «L’ordine #12345 è in spedizione. Arriverà lunedì.»
    Utente: «Posso aggiungere un nuovo articolo prima che venga spedito?»
    Assistente: «La spedizione è già stata avviata, quindi non è possibile modificare quest’ordine. Vuoi creare un nuovo ordine?»

#### - Guida Turistica
Obiettivo: Fornire consigli sui luoghi da visitare, itinerari di viaggio o informazioni pratiche (orari di apertura, costi, trasporti).
Esempio:

    Utente: «Cosa posso visitare in Toscana in due giorni?»
    Assistente: «Puoi dedicare il primo giorno a Firenze, visitando il Duomo, gli Uffizi e Ponte Vecchio. Il secondo giorno ti consiglio Pisa, per ammirare la Torre Pendente, e Lucca con le sue mura storiche.»
    Utente: «Quanto costa l’ingresso agli Uffizi?»
    Assistente: «Il biglietto standard costa circa 20€, ma ci sono riduzioni per studenti e over 65.»

#### - Servizio di Prenotazione
Obiettivo: Aiutare l’utente a prenotare un ristorante, un volo o un albergo, gestendo disponibilità e preferenze.
Esempio:

    Utente: «Vorrei prenotare un tavolo per 4 persone in un ristorante vegetariano, stasera alle 20.»
    Assistente: «Ho trovato tre ristoranti vegetariani nella tua zona con disponibilità per le 20. Ecco i nomi e le valutazioni: ... Vuoi procedere con la prenotazione?»
    Utente: «Sì, prenota il ristorante La Ghirlanda.»
    Assistente: «Fatto! Ho prenotato un tavolo per 4 alle 20. Ti ho inviato un’e-mail di conferma.»